## Цель проекта

* Поле Bucket не должно быть пустыми. Необходимо его заполнить и/или написать код, который будет это делать на основании значений других полей.
* Поля ADDR_FACT_FLG, ADDR_WORK_FLG, ADDR_REG_FLG не должны быть пустыми. Необходимо их заполнить и/или написать код, который будет это делать на основании значений других полей.
* Объяснить своими словами смысловое наполнение полей DEBTOR_ID, ASSET_ID, INITIAL_LOAN_AMT, если BIRTH_DT - это дата рождения
* Написать код для новго поля "возраст" (полных лет)
* В поле "WORK_START_DT" есть не корректные строки. Какие? Написать код, который их исправит (если возможно).
* Данная выгрузка сделана по состоянию на какую-то дату в прошлом. На какую?
* Заполнить недостающие значения поля EXPIRE_DAYS


### Импортируем библиотеки и изучим данные

In [253]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta

In [254]:
df = pd.read_excel("C:\\Users\\torop\\Downloads\\correction_of_data.XLSX", sheet_name=2)

In [255]:
df.head()

,,DEBTOR_ID,ASSET_ID,BUCKET,EXPIRE_DT,EXPIRE_DAYS,CURRENCY_CD,ADDR_FACT_FLG,ADDR_WORK_FLG,ADDR_REG_FLG,POST_FACT,POST_WORK,POST_REG,BIRTH_DT,WORK_START_DT,INITIAL_LOAN_AMT
0,28,1-T8PDBLT,1-T8PDBM7,1021-1050,2015-07-20,1047.0,RUR,0.0,1.0,1.0,NaN,142032,115372,1961-02-20,2017-12-08,33623.60
1,31,1-T5CQ4M4,1-T5CS1JC,1021-1050,2015-07-20,NaN,RUR,1.0,1.0,1.0,675000,675000,676716,1979-05-01,2017-11-23,16948.82
2,37,1-B0H67H0,1-T8P88NF,1021-1050,2015-07-20,NaN,RUR,1.0,1.0,1.0,630554,630000,630550,1985-01-28,2017-12-08,150000.00
3,38,1-OTX24JS,1-T8PCGLI,1021-1050,2015-07-20,NaN,RUR,1.0,1.0,1.0,308036,308001,309900,1989-07-09,2017-12-08,75917.61
4,39,1-T5CJS6U,1-T5CN27H,1021-1050,2015-07-20,NaN,RUR,0.0,1.0,1.0,NaN,666034,453501,1975-07-02,2017-11-23,106872.69


Распишем значения каждого столбца:
* DEBTOR_ID - ID должника
* ASSET_ID - id имущества 
* BUCKET - корзина по количеству дней просточки
* EXPIRE_DT - дата истечения срока
* EXPIRE_DAYS - сколько дней просрочки
* CURRENCY_CD - валюта займа
* ADDR_FACT_FLG - наличие адреса фактического проживания
* ADDR_WORK_FLG - наличие адреса работы
* ADDR_REG_FLG - наличие адреса регистрации
* POST_FACT - почтовый индекс адреса фактического проживания
* POST_WORK - почтовый индекс адреса работы
* POST_REG - почтовый индекс адреса регистрации
* BIRTH_DT - Дата рождения
* WORK_START_DT - Дата начала работы
* INITIAL_LOAN_AMT - Первоначальный заем

In [256]:
def preview(df):
    print('Первые 5 строк')
    display(df.head(10))
    print('-'*28)
    print('Распределение')
    print(df.describe().T)
    print('-'*28)
    print('Общая информация')
    print(df.info())
    print('-'*28)
    print('Количество пропусков')
    print(df.isna().sum())
    print('-'*28)
    print('Количество дубликатов:', df.duplicated().sum())

In [257]:
preview(df)

Первые 5 строк


,,DEBTOR_ID,ASSET_ID,BUCKET,EXPIRE_DT,EXPIRE_DAYS,CURRENCY_CD,ADDR_FACT_FLG,ADDR_WORK_FLG,ADDR_REG_FLG,POST_FACT,POST_WORK,POST_REG,BIRTH_DT,WORK_START_DT,INITIAL_LOAN_AMT
0,28,1-T8PDBLT,1-T8PDBM7,1021-1050,2015-07-20,1047.0,RUR,0.0,1.0,1.0,NaN,142032,115372,1961-02-20,2017-12-08,33623.60
1,31,1-T5CQ4M4,1-T5CS1JC,1021-1050,2015-07-20,NaN,RUR,1.0,1.0,1.0,675000,675000,676716,1979-05-01,2017-11-23,16948.82
2,37,1-B0H67H0,1-T8P88NF,1021-1050,2015-07-20,NaN,RUR,1.0,1.0,1.0,630554,630000,630550,1985-01-28,2017-12-08,150000.00
3,38,1-OTX24JS,1-T8PCGLI,1021-1050,2015-07-20,NaN,RUR,1.0,1.0,1.0,308036,308001,309900,1989-07-09,2017-12-08,75917.61
4,39,1-T5CJS6U,1-T5CN27H,1021-1050,2015-07-20,NaN,RUR,0.0,1.0,1.0,NaN,666034,453501,1975-07-02,2017-11-23,106872.69
5,65,1-HIKX1KS,1-T5CPIAQ,1021-1050,2015-07-20,1047.0,RUR,1.0,1.0,1.0,188349,195027,188349,1981-03-26,2017-11-23,56923.03
6,95,1-L6V5UND,1-T5CMUSJ,1021-1050,2015-08-06,1030.0,RUR,1.0,1.0,1.0,301790,301790,301790,1986-07-23,2017-11-23,51500.00
7,96,1-T5CIC96,1-T5CIC9K,1021-1050,2015-08-04,1032.0,RUR,0.0,1.0,1.0,NaN,656004,656056,1962-08-21,2017-11-23,750000.00
8,146,1-T5COVAO,1-T5COVR8,1021-1050,2015-07-20,1047.0,RUR,0.0,0.0,1.0,NaN,NaN,462770,1970-06-08,2017-11-23,450000.00
9,147,1-C4RH12C,1-UHE1BLF,1021-1050,2015-08-03,1033.0,RUR,1.0,1.0,1.0,453837,453838,453837,1974-02-07,2015-05-29,NaN


----------------------------
Распределение
                  count           mean            std    min        25%  \
                  150.0      75.500000      43.445368    1.0     38.250   
EXPIRE_DAYS       128.0    1175.171875     179.942690  502.0   1091.000   
ADDR_FACT_FLG     146.0       0.650685       0.478395    0.0      0.000   
ADDR_WORK_FLG     143.0       0.909091       0.288490    0.0      1.000   
ADDR_REG_FLG      149.0       0.973154       0.162177    0.0      1.000   
INITIAL_LOAN_AMT  112.0  178306.524107  284674.990211    0.0  17137.955   

                        50%        75%        max  
                     75.500     112.75      150.0  
EXPIRE_DAYS        1197.000    1283.50     1494.0  
ADDR_FACT_FLG         1.000       1.00        1.0  
ADDR_WORK_FLG         1.000       1.00        1.0  
ADDR_REG_FLG          1.000       1.00        1.0  
INITIAL_LOAN_AMT  68229.285  206633.00  1500000.0  
----------------------------
Общая информация
<class 'pandas.core.f

## Заполним пропуски

Проверим пропуски в столбце BUCKET

In [258]:
print(df['BUCKET'].unique())

['1021-1050' '1051-1080' nan '1081-1110' '1111-1140' '1141-1170'
 '1171-1200' '1201-1230' '1231-1260' '1261-1290' '1291-1320' '1321+'
 '481-510' '571-600' '691-720' '751-780' '811-840' '841-870' '901-930'
 '931-960' '961-990' '991-1020']


Корзины идут от 481 до 1321+ с интервалом 30. Не хватает корзин 541-570, 601-630, 631-660, 661-690, 721-750, 781-810, 871-900

В столбце EXPIRE_DAYS указано колличество полный дней просрочки, а значит мы можем вычислить дату составления датафрейма и заполнить недостающие значения.

In [259]:
Begindatestring = "2015-07-20" #возьмем дату из первой строки
Begindate = datetime.strptime(Begindatestring, "%Y-%m-%d")

End_date = Begindate + timedelta(days=1047) #добавим к дате срок просрочки в дняз
  
print("Дата составления таблицы", End_date)

Дата составления таблицы 2018-06-01 00:00:00


Заполним недостающие значения

In [260]:
df['EXPIRE_DAYS'] = End_date - df['EXPIRE_DT'] #посчитаем количество дней
df['EXPIRE_DAYS'] = df['EXPIRE_DAYS'].dt.days.astype('int16') #приведем к формату int

In [261]:
df.head(20) #проверим

,,DEBTOR_ID,ASSET_ID,BUCKET,EXPIRE_DT,EXPIRE_DAYS,CURRENCY_CD,ADDR_FACT_FLG,ADDR_WORK_FLG,ADDR_REG_FLG,POST_FACT,POST_WORK,POST_REG,BIRTH_DT,WORK_START_DT,INITIAL_LOAN_AMT
0,28,1-T8PDBLT,1-T8PDBM7,1021-1050,2015-07-20,1047,RUR,0.0,1.0,1.0,NaN,142032,115372,1961-02-20,2017-12-08,33623.60
1,31,1-T5CQ4M4,1-T5CS1JC,1021-1050,2015-07-20,1047,RUR,1.0,1.0,1.0,675000,675000,676716,1979-05-01,2017-11-23,16948.82
2,37,1-B0H67H0,1-T8P88NF,1021-1050,2015-07-20,1047,RUR,1.0,1.0,1.0,630554,630000,630550,1985-01-28,2017-12-08,150000.00
3,38,1-OTX24JS,1-T8PCGLI,1021-1050,2015-07-20,1047,RUR,1.0,1.0,1.0,308036,308001,309900,1989-07-09,2017-12-08,75917.61
4,39,1-T5CJS6U,1-T5CN27H,1021-1050,2015-07-20,1047,RUR,0.0,1.0,1.0,NaN,666034,453501,1975-07-02,2017-11-23,106872.69
5,65,1-HIKX1KS,1-T5CPIAQ,1021-1050,2015-07-20,1047,RUR,1.0,1.0,1.0,188349,195027,188349,1981-03-26,2017-11-23,56923.03
6,95,1-L6V5UND,1-T5CMUSJ,1021-1050,2015-08-06,1030,RUR,1.0,1.0,1.0,301790,301790,301790,1986-07-23,2017-11-23,51500.00
7,96,1-T5CIC96,1-T5CIC9K,1021-1050,2015-08-04,1032,RUR,0.0,1.0,1.0,NaN,656004,656056,1962-08-21,2017-11-23,750000.00
8,146,1-T5COVAO,1-T5COVR8,1021-1050,2015-07-20,1047,RUR,0.0,0.0,1.0,NaN,NaN,462770,1970-06-08,2017-11-23,450000.00
9,147,1-C4RH12C,1-UHE1BLF,1021-1050,2015-08-03,1033,RUR,1.0,1.0,1.0,453837,453838,453837,1974-02-07,2015-05-29,NaN


Теперь стало очевидно, что столбец BUCKET означает промежуток дней просрочки на дату выгрузки таблицы. И теперь мы можем заполнить пропуски.

Напишем функцию для автоматического заполнения поля BUCKET

In [262]:
def f(df):
    if df['EXPIRE_DAYS'] >= 481 and df['EXPIRE_DAYS'] <=510:
        val = '481-510'
    elif df['EXPIRE_DAYS'] >= 511 and df['EXPIRE_DAYS'] <=540:
        val = '511-540'
    elif df['EXPIRE_DAYS'] >= 541 and df['EXPIRE_DAYS'] <=570:
        val = '541-570'
    elif df['EXPIRE_DAYS'] >= 571 and df['EXPIRE_DAYS'] <=600:
        val = '571-600'
    elif df['EXPIRE_DAYS'] >= 601 and df['EXPIRE_DAYS'] <=630:
        val = '601-630'
    elif df['EXPIRE_DAYS'] >= 631 and df['EXPIRE_DAYS'] <=660:
        val = '631-660'
    elif df['EXPIRE_DAYS'] >= 661 and df['EXPIRE_DAYS'] <=690:
        val = '661-690'
    elif df['EXPIRE_DAYS'] >= 691 and df['EXPIRE_DAYS'] <=720:
        val = '691-720'
    elif df['EXPIRE_DAYS'] >= 721 and df['EXPIRE_DAYS'] <=750:
        val = '721-750'
    elif df['EXPIRE_DAYS'] >= 751 and df['EXPIRE_DAYS'] <=780:
        val = '751-780'
    elif df['EXPIRE_DAYS'] >= 781 and df['EXPIRE_DAYS'] <=810:
        val = '781-810'
    elif df['EXPIRE_DAYS'] >= 811 and df['EXPIRE_DAYS'] <=840:
        val = '811-840'
    elif df['EXPIRE_DAYS'] >= 841 and df['EXPIRE_DAYS'] <=870:
        val = '841-870'
    elif df['EXPIRE_DAYS'] >= 871 and df['EXPIRE_DAYS'] <=900:
        val = '871-900'   
    elif df['EXPIRE_DAYS'] >= 901 and df['EXPIRE_DAYS'] <=930:
        val = '901-930'
    elif df['EXPIRE_DAYS'] >= 931 and df['EXPIRE_DAYS'] <=960:
        val = '931-960'
    elif df['EXPIRE_DAYS'] >= 961 and df['EXPIRE_DAYS'] <=990:
        val = '961-990'
    elif df['EXPIRE_DAYS'] >= 991 and df['EXPIRE_DAYS'] <=1020:
        val = '991-1020'
    elif df['EXPIRE_DAYS'] >= 1021 and df['EXPIRE_DAYS'] <=1050:
        val = '1021-1050'
    elif df['EXPIRE_DAYS'] >= 1051 and df['EXPIRE_DAYS'] <=1080:
        val = '1051-1080'
    elif df['EXPIRE_DAYS'] >= 1081 and df['EXPIRE_DAYS'] <=1110:
        val = '1081-1110'
    elif df['EXPIRE_DAYS'] >= 1111 and df['EXPIRE_DAYS'] <=1140:
        val = '1111-1140'
    elif df['EXPIRE_DAYS'] >= 1141 and df['EXPIRE_DAYS'] <=1170:
        val = '1141-1170'
    elif df['EXPIRE_DAYS'] >= 1171 and df['EXPIRE_DAYS'] <=1200:
        val = '1171-1200'
    elif df['EXPIRE_DAYS'] >= 1201 and df['EXPIRE_DAYS'] <=1230:
        val = '1201-1230'
    elif df['EXPIRE_DAYS'] >= 1231 and df['EXPIRE_DAYS'] <=1260:
        val = '1231-1260'
    elif df['EXPIRE_DAYS'] >= 1261 and df['EXPIRE_DAYS'] <=1290:
        val = '1261-1290'
    elif df['EXPIRE_DAYS'] >= 1291 and df['EXPIRE_DAYS'] <=1320:
        val = '1291-1320'
    elif df['EXPIRE_DAYS'] >= 1321:
        val = '1321+'
    else :
        val = 'no'
    return val

In [263]:
df['BUCKET'] = df.apply (f, axis=1)

In [264]:
df.head() #проверим

,,DEBTOR_ID,ASSET_ID,BUCKET,EXPIRE_DT,EXPIRE_DAYS,CURRENCY_CD,ADDR_FACT_FLG,ADDR_WORK_FLG,ADDR_REG_FLG,POST_FACT,POST_WORK,POST_REG,BIRTH_DT,WORK_START_DT,INITIAL_LOAN_AMT
0,28,1-T8PDBLT,1-T8PDBM7,1021-1050,2015-07-20,1047,RUR,0.0,1.0,1.0,NaN,142032,115372,1961-02-20,2017-12-08,33623.60
1,31,1-T5CQ4M4,1-T5CS1JC,1021-1050,2015-07-20,1047,RUR,1.0,1.0,1.0,675000,675000,676716,1979-05-01,2017-11-23,16948.82
2,37,1-B0H67H0,1-T8P88NF,1021-1050,2015-07-20,1047,RUR,1.0,1.0,1.0,630554,630000,630550,1985-01-28,2017-12-08,150000.00
3,38,1-OTX24JS,1-T8PCGLI,1021-1050,2015-07-20,1047,RUR,1.0,1.0,1.0,308036,308001,309900,1989-07-09,2017-12-08,75917.61
4,39,1-T5CJS6U,1-T5CN27H,1021-1050,2015-07-20,1047,RUR,0.0,1.0,1.0,NaN,666034,453501,1975-07-02,2017-11-23,106872.69


In [265]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0                     150 non-null    int64         
 1   DEBTOR_ID         150 non-null    object        
 2   ASSET_ID          150 non-null    object        
 3   BUCKET            150 non-null    object        
 4   EXPIRE_DT         150 non-null    datetime64[ns]
 5   EXPIRE_DAYS       150 non-null    int16         
 6   CURRENCY_CD       150 non-null    object        
 7   ADDR_FACT_FLG     146 non-null    float64       
 8   ADDR_WORK_FLG     143 non-null    float64       
 9   ADDR_REG_FLG      149 non-null    float64       
 10  POST_FACT         96 non-null     object        
 11  POST_WORK         135 non-null    object        
 12  POST_REG          141 non-null    object        
 13  BIRTH_DT          149 non-null    datetime64[ns]
 14  WORK_START_DT     148 non-

Заполним пропущенные значения в столбце POST_FACT в соответствии со значениями в столбце ADDR_FACT_FLG. Если клиент не указал адвес фактического проживания, ADDR_FACT_FLG = 0, то проставим значения "НЕ УКАЗАНО", но бывает так что клиент указал адрес фактического проживания, но индекс по какой то причине не определился, то в этих полях проставим значения 'НЕ ОПРЕДЕЛЕНО'. По такой же логике заполним пропуски в столбцах с адресом работы и регистрации

In [266]:
df['POST_FACT'] = df['POST_FACT'].fillna(df['ADDR_FACT_FLG'])
df['POST_WORK'] = df['POST_WORK'].fillna(df['ADDR_WORK_FLG'])
df['POST_REG'] = df['POST_REG'].fillna(df['ADDR_REG_FLG'])

In [267]:
df.loc[df["POST_FACT"] == 1.0, 'POST_FACT'] = 'НЕ ОПРЕДЕЛЕНО'
df.loc[df["POST_WORK"] == 1.0, 'POST_WORK'] = 'НЕ ОПРЕДЕЛЕНО'
df.loc[df["POST_REG"] == 1.0, 'POST_REG'] = 'НЕ ОПРЕДЕЛЕНО'

In [268]:
df.loc[df["POST_FACT"] == 0.0, 'POST_FACT'] = 'НЕ УКАЗАНО'
df.loc[df["POST_WORK"] == 0.0, 'POST_WORK'] = 'НЕ УКАЗАНО'
df.loc[df["POST_REG"] == 0.0, 'POST_REG'] = 'НЕ УКАЗАНО'

В обратную сторону так же

In [269]:
df['ADDR_FACT_FLG'] = df['ADDR_FACT_FLG'].fillna(df['POST_FACT'])
df['ADDR_WORK_FLG'] = df['ADDR_WORK_FLG'].fillna(df['POST_WORK'])
df['ADDR_REG_FLG'] = df['ADDR_REG_FLG'].fillna(df['POST_REG'])

In [270]:
df.loc[df["ADDR_FACT_FLG"] == 'НЕ ОПРЕДЕЛЕНО', 'ADDR_FACT_FLG'] = 1.0
df.loc[df["ADDR_WORK_FLG"] == 'НЕ ОПРЕДЕЛЕНО', 'ADDR_WORK_FLG'] = 1.0
df.loc[df["ADDR_REG_FLG"] == 'НЕ ОПРЕДЕЛЕНО', 'ADDR_REG_FLG'] = 1.0

In [271]:
df.loc[df["ADDR_FACT_FLG"] != 0.0, 'ADDR_FACT_FLG'] = 1.0
df.loc[df["ADDR_WORK_FLG"] != 0.0, 'ADDR_WORK_FLG'] = 1.0
df.loc[df["ADDR_REG_FLG"] != 0.0, 'ADDR_REG_FLG'] = 1.0

In [272]:
df.loc[df["ADDR_FACT_FLG"] == 'НЕ УКАЗАНО', 'ADDR_FACT_FLG'] = 0.0
df.loc[df["ADDR_WORK_FLG"] == 'НЕ УКАЗАНО', 'ADDR_WORK_FLG'] = 0.0
df.loc[df["ADDR_REG_FLG"] == 'НЕ УКАЗАНО', 'ADDR_REG_FLG'] = 0.0

Проверим

In [273]:
df.head(40)

,,DEBTOR_ID,ASSET_ID,BUCKET,EXPIRE_DT,EXPIRE_DAYS,CURRENCY_CD,ADDR_FACT_FLG,ADDR_WORK_FLG,ADDR_REG_FLG,POST_FACT,POST_WORK,POST_REG,BIRTH_DT,WORK_START_DT,INITIAL_LOAN_AMT
0,28,1-T8PDBLT,1-T8PDBM7,1021-1050,2015-07-20,1047,RUR,0.0,1.0,1.0,НЕ УКАЗАНО,142032,115372,1961-02-20,2017-12-08,33623.60
1,31,1-T5CQ4M4,1-T5CS1JC,1021-1050,2015-07-20,1047,RUR,1.0,1.0,1.0,675000,675000,676716,1979-05-01,2017-11-23,16948.82
2,37,1-B0H67H0,1-T8P88NF,1021-1050,2015-07-20,1047,RUR,1.0,1.0,1.0,630554,630000,630550,1985-01-28,2017-12-08,150000.00
3,38,1-OTX24JS,1-T8PCGLI,1021-1050,2015-07-20,1047,RUR,1.0,1.0,1.0,308036,308001,309900,1989-07-09,2017-12-08,75917.61
4,39,1-T5CJS6U,1-T5CN27H,1021-1050,2015-07-20,1047,RUR,0.0,1.0,1.0,НЕ УКАЗАНО,666034,453501,1975-07-02,2017-11-23,106872.69
5,65,1-HIKX1KS,1-T5CPIAQ,1021-1050,2015-07-20,1047,RUR,1.0,1.0,1.0,188349,195027,188349,1981-03-26,2017-11-23,56923.03
6,95,1-L6V5UND,1-T5CMUSJ,1021-1050,2015-08-06,1030,RUR,1.0,1.0,1.0,301790,301790,301790,1986-07-23,2017-11-23,51500.00
7,96,1-T5CIC96,1-T5CIC9K,1021-1050,2015-08-04,1032,RUR,0.0,1.0,1.0,НЕ УКАЗАНО,656004,656056,1962-08-21,2017-11-23,750000.00
8,146,1-T5COVAO,1-T5COVR8,1021-1050,2015-07-20,1047,RUR,0.0,0.0,1.0,НЕ УКАЗАНО,НЕ УКАЗАНО,462770,1970-06-08,2017-11-23,450000.00
9,147,1-C4RH12C,1-UHE1BLF,1021-1050,2015-08-03,1033,RUR,1.0,1.0,1.0,453837,453838,453837,1974-02-07,2015-05-29,NaN


In [274]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0                     150 non-null    int64         
 1   DEBTOR_ID         150 non-null    object        
 2   ASSET_ID          150 non-null    object        
 3   BUCKET            150 non-null    object        
 4   EXPIRE_DT         150 non-null    datetime64[ns]
 5   EXPIRE_DAYS       150 non-null    int16         
 6   CURRENCY_CD       150 non-null    object        
 7   ADDR_FACT_FLG     150 non-null    object        
 8   ADDR_WORK_FLG     150 non-null    object        
 9   ADDR_REG_FLG      150 non-null    float64       
 10  POST_FACT         149 non-null    object        
 11  POST_WORK         149 non-null    object        
 12  POST_REG          149 non-null    object        
 13  BIRTH_DT          149 non-null    datetime64[ns]
 14  WORK_START_DT     148 non-

Отлично! Все пропуски, которые возможно было заполнить, мы заполнили

## Добавим поле с возврастом (полных лет)

In [275]:
df['AGE'] = End_date - df['BIRTH_DT']

In [276]:
df['AGE'] = df['AGE'].astype('timedelta64[Y]')

In [277]:
df.head(150)#проверим

,,DEBTOR_ID,ASSET_ID,BUCKET,EXPIRE_DT,EXPIRE_DAYS,CURRENCY_CD,ADDR_FACT_FLG,ADDR_WORK_FLG,ADDR_REG_FLG,POST_FACT,POST_WORK,POST_REG,BIRTH_DT,WORK_START_DT,INITIAL_LOAN_AMT,AGE
0,28,1-T8PDBLT,1-T8PDBM7,1021-1050,2015-07-20,1047,RUR,0.0,1.0,1.0,НЕ УКАЗАНО,142032,115372,1961-02-20,2017-12-08,33623.60,57.0
1,31,1-T5CQ4M4,1-T5CS1JC,1021-1050,2015-07-20,1047,RUR,1.0,1.0,1.0,675000,675000,676716,1979-05-01,2017-11-23,16948.82,39.0
2,37,1-B0H67H0,1-T8P88NF,1021-1050,2015-07-20,1047,RUR,1.0,1.0,1.0,630554,630000,630550,1985-01-28,2017-12-08,150000.00,33.0
3,38,1-OTX24JS,1-T8PCGLI,1021-1050,2015-07-20,1047,RUR,1.0,1.0,1.0,308036,308001,309900,1989-07-09,2017-12-08,75917.61,28.0
4,39,1-T5CJS6U,1-T5CN27H,1021-1050,2015-07-20,1047,RUR,0.0,1.0,1.0,НЕ УКАЗАНО,666034,453501,1975-07-02,2017-11-23,106872.69,42.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,10,1-T5CJOM2,1-T5CJP1Y,961-990,2015-09-21,984,RUR,1.0,1.0,1.0,634026,634034,634045,1956-01-30,2017-11-23,45589.81,62.0
146,110,1-BDSAUNV,1-UHDZV2I,991-1020,2015-08-25,1011,RUR,1.0,1.0,1.0,659305,НЕ ОПРЕДЕЛЕНО,659305,1969-01-15,2015-05-20,NaN,49.0
147,120,1-T8P6RSN,1-T8P6RT0,991-1020,2015-08-19,1017,RUR,0.0,1.0,1.0,НЕ УКАЗАНО,630511,630511,1992-07-25,2017-12-08,310000.00,25.0
148,131,1-6SBLIZ,1-T5CQ6W8,991-1020,2015-08-20,1016,RUR,1.0,1.0,1.0,НЕ ОПРЕДЕЛЕНО,658080,656065,1974-01-27,2017-11-23,15537.56,44.0


## Поправим не корректные значения в столбце WORK_START_DT

In [278]:
df['WORK_START_DT'].value_counts()

2017-11-23 00:00:00    30
2017-12-08 00:00:00    25
2017-10-19 00:00:00    15
2017-09-18 00:00:00    10
2017-04-04 00:00:00     2
2017-07-06 00:00:00     2
2014-12-29 00:00:00     2
2014-06-03 00:00:00     2
2016-10-27 00:00:00     2
2016-06-02 00:00:00     2
2014-08-16 00:00:00     2
2014-04-29 00:00:00     2
2016-02-01 00:00:00     2
2015-02-12 00:00:00     2
2016-04-03 00:00:00     2
2014-11-13 00:00:00     2
2015-02-10 00:00:00     2
2017-12-29 00:00:00     2
2016-11-08 00:00:00     2
2016-08-02 00:00:00     2
2016-10-07 00:00:00     2
2014-06-05 00:00:00     1
2017-05-03 00:00:00     1
2018-04-21 07:52:59     1
2015-03-30 00:00:00     1
2017-07-07 00:00:00     1
2018-04-12 07:12:18     1
2015-05-29 00:00:00     1
2016-05-02 00:00:00     1
2014-12-18 00:00:00     1
2014-06-02 00:00:00     1
2014-06-19 00:00:00     1
2015-01-17 00:00:00     1
2017-04-06 00:00:00     1
2014-11-24 00:00:00     1
2014-10-21 00:00:00     1
2014-04-27 00:00:00     1
2014-08-04 00:00:00     1
2016-08-16 0

На четырех строках проставилось время, приведем их к единому виду убрав время

In [279]:
df['WORK_START_DT'] = pd.to_datetime(df['WORK_START_DT']).dt.date

In [280]:
df['WORK_START_DT'].value_counts()

2017-11-23    30
2017-12-08    25
2017-10-19    15
2017-09-18    10
2017-04-04     2
2017-07-06     2
2014-12-29     2
2014-06-03     2
2016-10-27     2
2016-06-02     2
2014-08-16     2
2014-04-29     2
2016-02-01     2
2015-02-12     2
2016-04-03     2
2014-11-13     2
2015-02-10     2
2017-12-29     2
2016-11-08     2
2016-08-02     2
2016-10-07     2
2014-06-05     1
2017-05-03     1
2018-04-21     1
2015-03-30     1
2017-07-07     1
2018-04-12     1
2015-05-29     1
2016-05-02     1
2014-12-18     1
2014-06-02     1
2014-06-19     1
2015-01-17     1
2017-04-06     1
2014-11-24     1
2014-10-21     1
2014-04-27     1
2014-08-04     1
2016-08-16     1
2018-03-19     1
2015-03-04     1
2016-12-29     1
2015-02-09     1
2015-03-17     1
2015-08-03     1
2016-11-03     1
2015-02-11     1
2015-06-30     1
2014-09-29     1
2016-09-13     1
2014-12-04     1
2014-05-30     1
2018-03-10     1
2014-06-21     1
2015-05-20     1
Name: WORK_START_DT, dtype: int64

In [281]:
df.head()#проверим

,,DEBTOR_ID,ASSET_ID,BUCKET,EXPIRE_DT,EXPIRE_DAYS,CURRENCY_CD,ADDR_FACT_FLG,ADDR_WORK_FLG,ADDR_REG_FLG,POST_FACT,POST_WORK,POST_REG,BIRTH_DT,WORK_START_DT,INITIAL_LOAN_AMT,AGE
0,28,1-T8PDBLT,1-T8PDBM7,1021-1050,2015-07-20,1047,RUR,0.0,1.0,1.0,НЕ УКАЗАНО,142032,115372,1961-02-20,2017-12-08,33623.60,57.0
1,31,1-T5CQ4M4,1-T5CS1JC,1021-1050,2015-07-20,1047,RUR,1.0,1.0,1.0,675000,675000,676716,1979-05-01,2017-11-23,16948.82,39.0
2,37,1-B0H67H0,1-T8P88NF,1021-1050,2015-07-20,1047,RUR,1.0,1.0,1.0,630554,630000,630550,1985-01-28,2017-12-08,150000.00,33.0
3,38,1-OTX24JS,1-T8PCGLI,1021-1050,2015-07-20,1047,RUR,1.0,1.0,1.0,308036,308001,309900,1989-07-09,2017-12-08,75917.61,28.0
4,39,1-T5CJS6U,1-T5CN27H,1021-1050,2015-07-20,1047,RUR,0.0,1.0,1.0,НЕ УКАЗАНО,666034,453501,1975-07-02,2017-11-23,106872.69,42.0


In [282]:
writer = pd.ExcelWriter('Corrected table.xlsx', engine='xlsxwriter') 
df.to_excel(writer, sheet_name='AnotherSheet')
writer.save() #сохраним исправленную таблицу в формате xlsx